In [472]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import Sequential,Model
from keras.layers import Concatenate,Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add, BatchNormalization
#from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import pylab
import cv2
import os

In [2]:
!wget https://zenodo.org/record/4048312/files/public_dataset.zip?download=1

--2020-10-29 17:50:29--  https://zenodo.org/record/4048312/files/public_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 951442487 (907M) [application/octet-stream]
Saving to: ‘public_dataset.zip?download=1’

public_dataset.zip? 100%[===================>] 907.37M  7.36MB/s    in 2m 20s  

2020-10-29 17:52:51 (6.48 MB/s) - ‘public_dataset.zip?download=1’ saved [951442487/951442487]



In [ ]:
!unzip public_dataset.zip

In [ ]:
!ls public_dataset 

In [7]:
datadir  = '/content/public_dataset/'
custpath = '/content/custom_dataset/'

# Discovering The Dataset

In [8]:
metadata = pd.read_csv(os.path.join(datadir,'metadata_compiled.csv'),header=0)
metadata.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,quality_1,cough_type_1,dyspnea_1,wheezing_1,stridor_1,choking_1,congestion_1,nothing_1,diagnosis_1,severity_1,quality_2,cough_type_2,dyspnea_2,wheezing_2,stridor_2,choking_2,congestion_2,nothing_2,diagnosis_2,severity_2,quality_3,dyspnea_3,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
metadata = metadata.loc[metadata['cough_detected'] >= 0.9][['uuid','status']]
metadata.dropna(inplace=True)

In [10]:
metadata.info()
metadata['status'].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6092 entries, 0 to 20066
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uuid    6092 non-null   object
 1   status  6092 non-null   object
dtypes: object(2)
memory usage: 142.8+ KB


array(['healthy', 'symptomatic', 'COVID-19'], dtype=object)

In [11]:
extradata = metadata.loc[metadata['status']=='COVID-19']
notradata = metadata.loc[metadata['status']!='COVID-19'][0:1000]
totdata   = pd.concat([extradata,notradata],ignore_index= True)
totdata   = totdata.sample(frac=1).reset_index(drop=True)
totdata

,uuid,status
0,d0900d06-18dd-4fa9-94a7-7dc1e4d0f125,COVID-19
1,21ca6739-176f-4e5e-a6c7-44ef4185094b,symptomatic
2,60ab1646-c540-4179-ae54-ca83767edff1,COVID-19
3,2a8a1ee6-1704-4638-ab3f-2b88ca2fc8ba,healthy
4,02e59170-07cd-49eb-8d1f-59f9b0a0bec7,healthy
...,...,...
1436,dba6a1e3-ce39-438e-af77-9b1ce00332de,COVID-19
1437,2e6ad653-27dc-47df-87da-92f67585e904,healthy
1438,077edc1c-7f85-41a3-941a-cac380f9243a,healthy
1439,21aee478-6d13-45ea-be4d-4f29fd244798,COVID-19


In [12]:
def feature_extractor(audiofile):
  try:
    audio,sr = librosa.load(os.path.join(datadir,audiofile+'.webm'))
    #For MFCCS 
    mfccs    = librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=39)
    mfccsscaled = np.mean(mfccs.T,axis=0)

    #Mel Spectogram
    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
    melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
    s_db     = librosa.power_to_db(melspec, ref=np.max)
    librosa.display.specshow(s_db)

    savepath = os.path.join(custpath,audiofile+'.png')
    pylab.savefig(savepath, bbox_inches=None, pad_inches=0)
    pylab.close()
  except:
    print('File cannot open')
    return None,None
  return mfccsscaled,savepath

features = []
imgpaths = []

In [13]:
for value in tqdm(totdata['uuid']):
  #value += '.webm'
  mfccs,savepath  = feature_extractor(value)
  features.append(mfccs)
  imgpaths.append(savepath)

  3%|▎         | 49/1441 [00:39<13:29,  1.72it/s]

File cannot open


  6%|▌         | 86/1441 [01:09<13:35,  1.66it/s]

File cannot open


 12%|█▏        | 169/1441 [02:19<14:21,  1.48it/s]

...

 99%|█████████▊| 1421/1441 [23:22<00:25,  1.28s/it]

File cannot open


 99%|█████████▉| 1431/1441 [23:34<00:10,  1.02s/it]

File cannot open


100%|██████████| 1441/1441 [23:47<00:00,  1.01it/s]


In [209]:
isnone = lambda x: x is not None
label  = lambda x: 1 if x == 'COVID-19' else 0
cast_x = list(map(isnone,features))
data_y = list(map(label,totdata['status']))

data_x = [features[i] for i in range(len(features)) if cast_x[i] == True]
data_xx= [imgpaths[i] for i in range(len(imgpaths)) if cast_x[i] == True]
data_y = [data_y[i] for i in range(len(features)) if cast_x[i] == True]

In [210]:
print(len(data_x))
print(len(data_y))

1397
1397


In [211]:
data_x = np.array(data_x)
data_y = np.array(data_y)

In [212]:
print(data_x.shape[1])

39


In [213]:
train_x = data_x[:int(0.8*len(data_x))]
train_y = data_y[:int(0.8*len(data_y))]

test_x  = data_x[int(0.8*len(data_x)):]
test_y  = data_y[int(0.8*len(data_x)):]

In [214]:
model = Sequential()
#model.add(Dropout(0.1))
model.add(Dense(units=512,activation='relu',input_dim=data_x.shape[1],kernel_initializer='GlorotUniform'))
model.add(Dropout(0.4))
model.add(Dense(units=256,activation='relu',kernel_initializer='GlorotUniform'))
model.add(Dropout(0.2))
model.add(Dense(units=2,activation='softmax'))

model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 512)               20480     
_________________________________________________________________
dropout_62 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_63 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 2)                 514       
Total params: 152,322
Trainable params: 152,322
Non-trainable params: 0
_________________________________________________________________


In [215]:
model.compile(
    optimizer = 'Adam',
    loss = 'CategoricalCrossentropy',
    metrics=['AUC']
)

In [216]:
print("Fit model on training data")
history = model.fit(
    train_x,
    keras.utils.to_categorical(train_y, num_classes=2),
    batch_size=48,
    epochs=120,
)


Fit model on training data
Epoch 1/120
24/24 [==============================] - 0s 3ms/step - loss: 12.5859 - auc: 0.5702
Epoch 2/120
24/24 [==============================] - 0s 3ms/step - loss: 6.6399 - auc: 0.5935
Epoch 3/120
24/24 [==============================] - 0s 4ms/step - loss: 4.4175 - auc: 0.5890
Epoch 4/120
24/24 [==============================] - 0s 3ms/step - loss: 2.7784 - auc: 0.6005
Epoch 5/120
24/24 [==============================] - 0s 3ms/step - loss: 2.2298 - auc: 0.5839
Epoch 6/120
24/24 [==============================] - 0s 3ms/step - loss: 1.4275 - auc: 0.6150
Epoch 7/120
24/24 [==============================] - 0s 3ms/step - loss: 1.0913 - auc: 0.6211
Epoch 8/120
24/24 [==============================] - 0s 3ms/step - loss: 0.9007 - auc: 0.6346
Epoch 9/120
24/24 [==============================] - 0s 3ms/step - loss: 0.8128 - auc: 0.6521
Epoch 10/120
24/24 [==============================] - 0s 3ms/step - loss: 0.7389 - auc: 0.6419
Epoch 11/120
24/24 [===========

In [218]:
model.evaluate(test_x,keras.utils.to_categorical(test_y, num_classes=2))

9/9 [==============================] - 0s 3ms/step - loss: 0.6141 - auc: 0.7098


[0.6140801310539246, 0.7098214626312256]

In [219]:
model.evaluate(train_x,keras.utils.to_categorical(train_y, num_classes=2))

35/35 [==============================] - 0s 3ms/step - loss: 0.6012 - auc: 0.7302


[0.6011629700660706, 0.7302420735359192]

In [99]:
op = cv2.imread('/content/custom_dataset/00039425-7f3a-42aa-ac13-834aaa2b6b92.png')
op.shape

(288, 432, 3)

---
# Second Model
---



In [484]:
secmodel = Sequential()
input_shape=(64,64, 3)#1st hidden layer
secmodel.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
secmodel.add(AveragePooling2D((2, 2), strides=(2,2)))
secmodel.add(BatchNormalization())
secmodel.add(Activation('relu'))

secmodel.add(Conv2D(64, (3, 3), padding="same"))
secmodel.add(AveragePooling2D((2, 2), strides=(2,2)))
secmodel.add(BatchNormalization())
secmodel.add(Activation('relu'))

secmodel.add(Conv2D(64, (3, 3), padding="same"))
secmodel.add(AveragePooling2D((2, 2), strides=(2,2)))
secmodel.add(BatchNormalization())
secmodel.add(Activation('relu'))

secmodel.add(Flatten())
secmodel.add(Dropout(rate=0.5))

secmodel.add(Dense(64,activation='relu'))
secmodel.add(BatchNormalization())

secmodel.add(Dropout(rate=0.5))
secmodel.add(Dense(2,activation='softmax'))
secmodel.summary()

Model: "sequential_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_217 (Conv2D)          (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_216 (Avera (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_98 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
activation_217 (Activation)  (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_218 (Conv2D)          (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_217 (Avera (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_99 (Batc (None, 7, 7, 64)        

In [485]:
train_x = data_xx[:int(0.8*len(data_xx))]
train_y = data_y[:int(0.8*len(data_y))]

test_x  = data_xx[int(0.8*len(data_xx)):]
test_y  = data_y[int(0.8*len(data_x)):]

'''
def custom_generator(imgfiles,labels,scale=255,target_size=(128,128),shuffle= False,batch_size=32):
  i  = 0
  length= len(imgfiles)

  while (True):
    if (i*batch_size >= length):
      return None,None
    imgs = []
    for dir in imgfiles[i*batch_size:(i+1)*batch_size]:
      img = cv2.imread(dir)
      img = cv2.resize(img,target_size,interpolation = cv2.INTER_CUBIC)
      img = img / scale
      imgs.append(img)
    
    imgs  = np.array(imgs)
    labels= labels[i*batch_size:(i+1)*batch_size]
    yield imgs, keras.utils.to_categorical(labels, num_classes=2)
    i +=1
'''
class CustomDataset(tf.keras.utils.Sequence):
  def __init__(self,imgfiles,labels,batch_size,target_size=(64,64),shuffle=False,scale=255,n_classes=2,n_channels=3):
    self.batch_size = batch_size
    self.dim        = target_size
    self.labels     = labels
    self.imgfiles   = imgfiles
    self.n_classes  = n_classes
    self.shuffle    = shuffle
    self.n_channels = n_channels
    self.scale      = scale

    self.c          = 0
    self.on_epoch_end()

  def __len__(self):
    # returns the number of batches
    return int(np.floor(len(self.imgfiles) / self.batch_size))

  def __getitem__(self, index):
    # returns one batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Generate data
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.imgfiles))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        # Store sample
        img   = cv2.imread(self.imgfiles[ID])
        img   = cv2.resize(img,self.dim,interpolation = cv2.INTER_CUBIC)
        
        '''
        if (self.c %5 ==1):
          img   = cv2.flip(img, 0)
        elif (self.c %5 ==2):
          img   = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)
        elif (self.c %5 ==3):
          img   = cv2.flip(img, -1)
        elif (self.c %5 ==4):
          img   = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
        '''  
        X[i,] = img / self.scale

        # Store class
        y[i] = self.labels[ID]

        self.c +=1
    return X, keras.utils.to_categorical(y, num_classes=self.n_classes)



#train_datagen = custom_generator(train_x,train_y)
#test_datagen  = custom_generator(test_x,test_y)
train_datagen = CustomDataset(train_x,train_y,batch_size=48,shuffle=True)
test_datagen  = CustomDataset(test_x,test_y,batch_size=48,shuffle=True)

In [486]:
secmodel.compile(
    optimizer = 'Adam',
    loss = 'CategoricalCrossentropy',
    metrics=['AUC']
)

In [487]:
print("Fit second model on training data")
sechistory = secmodel.fit_generator(
            train_datagen,
            steps_per_epoch=len(train_x)//48,
            epochs=20,)

Fit second model on training data
Epoch 1/20
23/23 [==============================] - 2s 107ms/step - loss: 1.1850 - auc: 0.5173
Epoch 2/20
23/23 [==============================] - 3s 109ms/step - loss: 0.9464 - auc: 0.5806
Epoch 3/20
23/23 [==============================] - 3s 109ms/step - loss: 0.9110 - auc: 0.5789
Epoch 4/20
23/23 [==============================] - 2s 109ms/step - loss: 0.8487 - auc: 0.6136
Epoch 5/20
23/23 [==============================] - 3s 110ms/step - loss: 0.7702 - auc: 0.6611
Epoch 6/20
23/23 [==============================] - 3s 110ms/step - loss: 0.7627 - auc: 0.6493
Epoch 7/20
23/23 [==============================] - 3s 109ms/step - loss: 0.7484 - auc: 0.6493
Epoch 8/20
23/23 [==============================] - 3s 110ms/step - loss: 0.6990 - auc: 0.6814
Epoch 9/20
23/23 [==============================] - 3s 110ms/step - loss: 0.6832 - auc: 0.6903
Epoch 10/20
23/23 [==============================] - 3s 110ms/step - loss: 0.6760 - auc: 0.6945
Epoch 11/20
23/

In [488]:
test_datagen  = CustomDataset(test_x,test_y,batch_size=48,shuffle=False)

secmodel.evaluate_generator(
  test_datagen    
)

[0.6267070770263672, 0.7203037738800049]

---
# Multiple Input Mixed Data Model
---



In [265]:
merge = Concatenate()([model.output,secmodel.output])
thirdmodel = Model(inputs=[model.input,secmodel.input], outputs=merge)

In [266]:
thirdmodel.compile(
    optimizer = 'Adam',
    loss = 'CategoricalCrossentropy',
    metrics=['AUC']
)

In [267]:
thirdmodel.evaluate_generator(
  test_datagen    
)

AssertionError: ignored